In [1]:
import requests
import pandas as pd

# Your API key for FoodData Central
api_key = 'uvc2RJDlyvc4xJFsFp7v9REGVi4Uq26il1UCEmXD'

# Base URL for FoodData Central
base_url = "https://api.nal.usda.gov/fdc/v1/foods/search"

# Foods to search for
foods = {"Lentils": 172420, 
         "Broccoli":2346366, 
         "Cauliflower":2346374, 
         "Mushrooms":2346370, 
         "Sweet Potato":168483,
         "Ginger": 169231,
         "Tumeric":172231,
         "Tahini":2343062,
         "Cider vinegar":173469,
         "Hemp Seeds":170148, 
         "Pumpkin Seeds":170556,
         "Olive Oil":171413,
         
         "Walnuts":170187, 
         "Macadamia":170178, 
         "Brazil nuts":170569,
         "Almonds":170567,
         "Sunflower seeds":170562,
         "Blueberries":2344771, 
         "Chia seeds":170554,
         "Flax seeds":2343063,
         "Pomegranate juice":167787,
         "Soy joghurt":2342919,
         "Cocoa powder":170657
         }

food_amounts = {"Lentils": 70, 
                "Broccoli":200, 
                "Cauliflower":100, 
                "Mushrooms":50, 
                "Sweet Potato":100,
                "Ginger": 3,
                "Tumeric":2,
                "Tahini":15,
                "Cider vinegar":10,
                "Hemp Seeds":30, 
                "Pumpkin Seeds":10,
                "Olive Oil":25,
                
                "Walnuts":25, 
                "Macadamia":20, 
                "Brazil nuts":7,
                "Almonds":10,
                "Sunflower seeds":10,
                "Blueberries":90, 
                "Chia seeds":20,
                "Flax seeds":4,
                "Pomegranate juice":50,
                "Soy joghurt":75,
                "Cocoa powder":20
                }

assert foods.keys() == food_amounts.keys()

# Nutrients of interest
nutrients_of_interest = {
    "Engery (kcal)": 1008,
    "Total Fat": 1004,
    "Saturated Fat": 1258,
    "Monounsaturated Fat": 1292,
    "Polyunsaturated Fat": 1293,
    "Cholesterol": 1253,
    "Carbohydrate": 1005,
    "Fiber": 1079,
    "Sugars": 2000,
    "Protein": 1003,
    
    "Calcium": 1087,
    "Iron": 1089,
    "Magnesium": 1090,
    "Phosphorus": 1091,
    "Potassium": 1092,
    "Sodium": 1093,
    "Zinc": 1095,
    "Copper": 1098,
    "Manganese": 1101,
    "Selenium": 1103,
    
    "Vitamin A": 1106,
    "Thiamin (B1)": 1165,
    "Riboflavin (B2)": 1166,
    "Niacin (B3)": 1167,
    "Pantothenic Acid (B5)": 1170,
    "Vitamin B6": 1175,
    "Folate (B9)": 1177,
    "Vitamin B12": 1178,
    "Vitamin C": 1162,
    "Vitamin D": 1110,
    "Vitamin E": 1109,
    "Vitamin K": 1185,
    
    "Choline": 1180,
}


daily_recommendations = {
    "Engery (kcal)": "/",
    "Total Fat": "/",
    "Saturated Fat": "/",
    "Monounsaturated Fat": "/",
    "Polyunsaturated Fat": "/",
    "Cholesterol": "/",
    "Carbohydrate": "/",
    "Fiber": "/",
    "Sugars": "/",
    "Protein": "/",
    
    "Calcium": 1000,
    "Iron": 8 ,
    "Magnesium": 400,
    "Phosphorus": 700,
    "Potassium": 3000,
    "Sodium": 1500 ,
    "Zinc": 11,
    "Copper": 0.9 ,
    "Manganese": 2.3,
    "Selenium": 55,
    
    "Vitamin A": 900 ,
    "Thiamin (B1)": 1.2,
    "Riboflavin (B2)": 1.3,
    "Niacin (B3)": 16,
    "Pantothenic Acid (B5)": 5,
    "Vitamin B6": 1.3 ,
    "Folate (B9)": 400 ,
    "Vitamin B12": 2.4,
    "Vitamin C": 90,
    "Vitamin D": 600,
    "Vitamin E": 15,
    "Vitamin K": 120 ,
    
    "Choline": 550,
}

assert nutrients_of_interest.keys() == daily_recommendations.keys()

In [2]:
# Function to get nutrient data for a given food ID
def get_nutrient_data(food_id, nutrients):
    url = f"https://api.nal.usda.gov/fdc/v1/food/{food_id}"
    params = {'api_key': api_key}
    response = requests.get(url, params=params).json()

    nutrient_data = {}
    for nutrient in nutrients:
        nutrient_data[nutrient] = None  # Default to None if nutrient not found
        if 'foodNutrients' in response:
            for item in response['foodNutrients']:
                if 'nutrient' in item and item['nutrient']['id'] == nutrients[nutrient]:
                    nutrient_data[nutrient] = item.get('amount')
                    break

    return nutrient_data


In [3]:
# Main script to compile nutrient data for each food
nutrient_table = pd.DataFrame(index=foods.keys(), columns=nutrients_of_interest.keys())
for name, id in foods.items():
    nutrients = get_nutrient_data(id, nutrients_of_interest)
    nutrient_table.loc[name] = nutrients
    
# scale every item to food amount
for name, amount in food_amounts.items():
    nutrient_table.loc[name] = nutrient_table.loc[name] * amount / 100


nutrient_table["Amount (g)"] = food_amounts.values()

In [4]:
# take only the nutrients of interest
nutrient_table_sub = nutrient_table.iloc[:,10:]

# move amount to first column
cols = nutrient_table_sub.columns.tolist()
cols = cols[-1:] + cols[:-1]
nutrient_table_sub = nutrient_table_sub[cols]

nutrient_table_sub.to_html("scripts/micros_table.html")
nutrient_table_sub

,Amount (g),Calcium,Iron,Magnesium,Phosphorus,Potassium,Sodium,Zinc,Copper,Manganese,...,Niacin (B3),Pantothenic Acid (B5),Vitamin B6,Folate (B9),Vitamin B12,Vitamin C,Vitamin D,Vitamin E,Vitamin K,Choline
Lentils,70,24.5,4.557,32.9,196.7,473.9,4.2,2.289,0.5278,0.9751,...,1.8235,1.498,0.378,335.3,0.0,3.15,0.0,0.343,3.5,67.48
Broccoli,200,96.0,1.44,44.0,140.0,632.0,76.0,0.88,0.122,NaN,...,1.264,NaN,0.358,116.0,0.0,161.6,NaN,0.32,212.0,39.0
Cauliflower,100,23.0,0.44,16.0,46.0,311.0,31.0,0.28,0.041,NaN,...,0.502,NaN,0.172,50.0,0.0,42.7,NaN,0.08,16.1,46.1
Mushrooms,50,3.0,0.145,6.0,58.0,232.0,3.5,0.31,0.2275,NaN,...,2.325,NaN,0.046,18.5,0.025,1.1,NaN,0.005,0.0,10.8
Sweet Potato,100,38.0,0.69,27.0,54.0,475.0,36.0,0.32,0.161,0.497,...,1.487,0.884,0.286,6.0,0.0,19.6,0.0,0.71,2.3,13.1
Ginger,3,0.48,0.018,1.29,1.02,12.45,0.39,0.0102,0.00678,0.00687,...,0.0225,0.00609,0.0048,0.33,0.0,0.15,0.0,0.0078,0.003,0.864
Tumeric,2,3.36,1.1,4.16,5.98,41.6,0.54,0.09,0.026,0.396,...,0.027,0.01084,0.00214,0.4,0.0,0.014,0.0,0.0886,0.268,0.984
Tahini,15,63.9,1.3425,14.25,109.8,62.1,17.25,0.693,0.2415,NaN,...,0.8175,NaN,0.02235,14.7,0.0,0.0,NaN,0.0375,0.0,3.87
Cider vinegar,10,0.7,0.02,0.5,0.8,7.3,0.5,0.004,0.0008,0.0249,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Hemp Seeds,30,21.0,2.385,210.0,495.0,360.0,1.5,2.97,0.48,2.28,...,2.76,NaN,0.18,33.0,NaN,0.15,NaN,0.24,NaN,NaN


In [5]:
# Summary table
summary_table = pd.DataFrame()
summary_table.index = nutrient_table_sub.columns[1:]
summary_table["Amount"] = nutrient_table_sub.sum(axis=0, skipna=True)[1:].values
summary_table["Daily recommendation"] = [item for item in daily_recommendations.values()][10:]
summary_table["Ratio"] = summary_table["Amount"] / summary_table["Daily recommendation"]
# format so that there are only two decimals
summary_table["Amount"] = summary_table["Amount"].map('{:,.1f}'.format)
summary_table["Daily recommendation"] = summary_table["Daily recommendation"].map('{:,.1f}'.format)
summary_table["Ratio"] = summary_table["Ratio"].map('{:,.2f}'.format)

summary_table.to_html("scripts/micros_table_summary.html")
summary_table

,Amount,Daily recommendation,Ratio
Calcium,628.2,"1,000.0",0.63
Iron,21.4,8.0,2.67
Magnesium,782.5,400.0,1.96
Phosphorus,"1,912.7",700.0,2.73
Potassium,"3,862.9","3,000.0",1.29
Sodium,214.8,"1,500.0",0.14
Zinc,13.5,11.0,1.23
Copper,4.0,0.9,4.42
Manganese,7.4,2.3,3.22
Selenium,188.8,55.0,3.43


In [10]:
# Macros only
macro_table = nutrient_table.iloc[:,:10]

macro_table["Amount (g)"] = nutrient_table["Amount (g)"]
# move amount to first column
cols = macro_table.columns.tolist()
cols = cols[-1:] + cols[:-1]
macro_table = macro_table[cols]

# add last row with sum
macro_table.loc["Sum"] = macro_table.sum(axis=0)

macro_table.to_html("scripts/macros_table.html")
macro_table

,Amount (g),Engery (kcal),Total Fat,Saturated Fat,Monounsaturated Fat,Polyunsaturated Fat,Cholesterol,Carbohydrate,Fiber,Sugars,Protein
Lentils,70,246.4,0.742,0.1078,0.1351,0.3682,0.0,44.345,7.49,1.421,17.241
Broccoli,200,82.0,0.7,0.082,0.022,0.036,0.0,13.06,5.0,2.92,5.36
Cauliflower,100,26.0,0.29,0.135,0.035,0.032,0.0,5.18,2.1,1.99,2.0
Mushrooms,50,19.5,0.215,0.031,0.0,0.1,0.0,2.58,0.6,1.24,1.81
Sweet Potato,100,90.0,0.15,0.052,0.002,0.092,0.0,20.71,3.3,6.48,2.01
Ginger,3,2.4,0.0225,0.00609,0.00462,0.00462,0.0,0.5331,0.06,0.051,0.0546
Tumeric,2,6.24,0.065,0.03676,0.00898,0.01512,0.0,1.3428,0.454,0.0642,0.1936
Tahini,15,89.25,8.07,1.1295,3.045,3.54,0.0,3.18,1.395,0.0735,2.55
Cider vinegar,10,2.1,0.0,0.0,0.0,0.0,0.0,0.093,0.0,0.04,0.0
Hemp Seeds,30,165.9,14.625,1.38,1.62,11.43,0.0,2.601,1.2,0.45,9.468
